# Kapitel 3: Dekoratoren & Professionelles Klassendesign
---

## **1. Das Fundament: Funktionen als "First-Class Citizens"**

### **Was bedeutet das?**

Bevor wir Dekoratoren verstehen können, müssen wir ein Kernprinzip von Python verinnerlichen: Funktionen sind Objekte erster Klasse ("First-Class Citizens"). Das bedeutet, sie können wie jede andere Variable (z.B. eine Zahl oder ein String) behandelt werden.

* **Eigenschaft 1: Einer Variable zuweisen:** Man kann eine Funktion einer Variable zuweisen und sie dann über diese Variable aufrufen.
* **Eigenschaft 2: Als Argument übergeben:** Man kann eine Funktion als Argument an eine andere Funktion übergeben.
* **Eigenschaft 3: Aus einer Funktion zurückgeben:** Eine Funktion kann eine andere Funktion erzeugen und zurückgeben.

---

### **Isoliertes Beispiel 1: Funktion einer Variable zuweisen**


In [20]:
def hallo_sagen():
    print("Hallo Welt!")

# Die Funktion 'hallo_sagen' wird der Variable 'gruss' zugewiesen.
# Wichtig: Ohne Klammern! Wir weisen die Funktion selbst zu, nicht ihr Ergebnis.
gruss = hallo_sagen

print(f"Der Typ von 'gruss' ist: {type(gruss)}")

# Jetzt können wir die Funktion über die neue Variable aufrufen
gruss()

Der Typ von 'gruss' ist: <class 'function'>
Hallo Welt!



---

### **Funktion als Argument übergeben**

Dies ist ein sehr häufiges Muster in Python. Wir schreiben eine Funktion, die eine andere Funktion als "Werkzeug" entgegennimmt.

In [21]:
def sag_hallo():
    return "Hallo"

def sag_auf_wiedersehen():
    return "Auf Wiedersehen"

def begruesse_person(gruss_funktion, name):
    # Hier wird die übergebene Funktion aufgerufen
    print(f"{gruss_funktion()}, {name}!")

# Wir übergeben die Funktion 'sag_hallo' als Argument
begruesse_person(sag_hallo, "Anna")

# Wir übergeben die Funktion 'sag_auf_wiedersehen' als Argument
begruesse_person(sag_auf_wiedersehen, "Ben")

Hallo, Anna!
Auf Wiedersehen, Ben!



---

### **Isoliertes Beispiel 3: Funktion zurückgeben & Closures**

Dies ist der wichtigste Schritt zum Verständnis von Dekoratoren. Eine Funktion kann eine andere, in ihr definierte Funktion, zurückgeben.

Die zurückgegebene innere Funktion "erinnert" sich an die Variablen aus der äußeren Funktion, in der sie erstellt wurde. Dieses Konzept nennt man **Closure**.

In [22]:
def erstelle_gruss_funktion(gruss_text):
    # Diese innere Funktion wird erstellt, aber noch nicht ausgeführt.
    # Sie hat Zugriff auf 'gruss_text' aus ihrem "Geburtsort".
    def gruss_funktion(name):
        return f"{gruss_text}, {name}!"
    
    # Die äußere Funktion gibt die innere Funktion zurück
    return gruss_funktion

# Wir erstellen zwei verschiedene "Gruß-Funktionen"
sage_hallo = erstelle_gruss_funktion("Hallo")
sage_guten_tag = erstelle_gruss_funktion("Guten Tag")

# Obwohl 'erstelle_gruss_funktion' bereits beendet ist,
# "erinnern" sich die zurückgegebenen Funktionen an ihren 'gruss_text'.
print(sage_hallo("Welt"))
print(sage_guten_tag("Python-Entwickler"))

Hallo, Welt!
Guten Tag, Python-Entwickler!


---
## **2. Dekoratoren von Grund auf entmystifizieren**

### **Theorie: Das "Warum" – Ein wiederkehrendes Problem**

Stellen Sie sich vor, Sie haben mehrere Funktionen in Ihrem Programm. Nun bekommen Sie die Aufgabe, für jede dieser Funktionen zu protokollieren, wann sie startet und wann sie endet.

**Die naive Lösung:** Sie könnten den Protokoll-Code in jede einzelne Funktion hineinkopieren:

In [23]:
def funktion_eins():
    print("[LOG] Funktion 'funktion_eins' startet...")
    # ... eigentlicher Code der Funktion ...
    print("[LOG] Funktion 'funktion_eins' beendet.")

def funktion_zwei():
    print("[LOG] Funktion 'funktion_zwei' startet...")
    # ... anderer Code ...
    print("[LOG] Funktion 'funktion_zwei' beendet.")

Das Problem hierbei ist offensichtlich: Es verstößt massiv gegen das **DRY-Prinzip (Don't Repeat Yourself)**. Bei jeder Änderung der Logging-Logik müssten Sie alle Funktionen anpassen. Das ist fehleranfällig und ineffizient.

**Die elegante Lösung:** Wir brauchen einen Weg, unsere Funktionen mit der Logging-Fähigkeit zu "dekorieren" oder "einzuhüllen", ohne ihren eigenen Code zu verändern. Genau das leisten Dekoratoren.

---

### **Wie Dekoratoren im Hintergrund funktionieren – Der manuelle Weg**

Bevor wir die `@`-Syntax verwenden, bauen wir einen Dekorator manuell zusammen. Das ist der wichtigste Schritt, um die "Magie" dahinter zu verstehen. Wir nutzen hier unser Wissen von vorhin: Funktionen sind Objekte, die man wie Variablen behandeln kann.

**Schritt A: Unsere ursprüngliche, saubere Funktion**
Dies ist die Funktion, der wir eine neue Fähigkeit hinzufügen wollen.

In [24]:
def sag_hallo():
    print("Hallo!")

**Schritt B: Die Dekorator-Funktion schreiben**
Ein Dekorator ist eine Funktion, die eine andere Funktion als Argument entgegennimmt und eine *neue*, modifizierte Funktion zurückgibt.

In [25]:
# 'einfacher_dekorator' nimmt eine Funktion (z.B. sag_hallo) als Argument
def einfacher_dekorator(original_funktion):
    
    # Im Inneren definieren wir eine neue Funktion, den "Wrapper".
    # Dieser Wrapper wird am Ende unsere Originalfunktion ersetzen.
    def wrapper():
        print("--- Code, der VOR der Originalfunktion ausgeführt wird. ---")
        
        # Hier rufen wir die Originalfunktion auf, die uns übergeben wurde.
        original_funktion()
        
        print("--- Code, der NACH der Originalfunktion ausgeführt wird. ---")
        
    # Der Dekorator gibt die neu erstellte Wrapper-Funktion zurück.
    return wrapper

**Schritt C: Den Dekorator manuell anwenden**
Jetzt kommt der entscheidende Teil. Wir rufen den Dekorator mit unserer Originalfunktion auf und weisen das Ergebnis (die `wrapper`-Funktion) wieder der ursprünglichen Variable zu.

In [26]:
# Wir übergeben 'sag_hallo' an den Dekorator.
# Der Dekorator gibt uns seine 'wrapper'-Funktion zurück.
# Wir überschreiben die alte 'sag_hallo'-Funktion mit diesem neuen Wrapper.
sag_hallo = einfacher_dekorator(sag_hallo)

# Was passiert, wenn wir jetzt sag_hallo() aufrufen?
# Wir rufen nicht mehr die ursprüngliche Funktion auf, sondern den Wrapper!
sag_hallo()

--- Code, der VOR der Originalfunktion ausgeführt wird. ---
Hallo!
--- Code, der NACH der Originalfunktion ausgeführt wird. ---


**Analyse:** Wir haben die Funktionalität von `sag_hallo` geändert, ohne ihren Quellcode anzufassen. Wir haben sie in eine andere Funktion "eingewickelt".

---

### **Theorie: Die `@`-Syntax – Reiner syntaktischer Zucker**

Python bietet für diesen Zuweisungsprozess (`sag_hallo = einfacher_dekorator(sag_hallo)`) eine schönere, deklarative Syntax: das `@`-Symbol.

Der folgende Code ist **100% identisch** mit dem manuellen Weg von oben:

In [27]:
@einfacher_dekorator
def sag_hallo_mit_at():
    print("Hallo mit @-Syntax!")

# Dieser Aufruf führt jetzt direkt den Wrapper aus.
sag_hallo_mit_at()

--- Code, der VOR der Originalfunktion ausgeführt wird. ---
Hallo mit @-Syntax!
--- Code, der NACH der Originalfunktion ausgeführt wird. ---


**Merksatz:** `@dekorator` über einer Funktion `func` ist nur eine Abkürzung für `func = dekorator(func)`.

---

### **Umgang mit Argumenten – `*args` und `**kwargs`**

Unser `einfacher_dekorator` hat ein Problem: Er funktioniert nur für Funktionen, die keine Argumente haben.

**Das Problem demonstriert:**


In [28]:
@einfacher_dekorator
def sag_etwas(text):
    print(text)
    
# Dieser Aufruf würde einen Fehler erzeugen:
# sag_etwas("Dies wird fehlschlagen") # TypeError: wrapper() takes 0 positional arguments but 1 was given

Der Fehler passiert, weil unser `wrapper()` keine Argumente entgegennimmt, wir aber versuchen, `sag_etwas` mit dem Argument `"Dies wird fehlschlagen"` aufzurufen.

**Die Lösung:** Wir müssen unseren Wrapper so gestalten, dass er eine beliebige Anzahl von Argumenten akzeptieren und diese an die Originalfunktion weiterleiten kann. Dafür sind `*args` und `**kwargs` da.

* **`*args` (Arguments):** Sammelt alle **positionsbasierten Argumente** in einem **Tupel**.
* **`**kwargs` (Keyword Arguments):** Sammelt alle **schlüsselwortbasierten Argumente** in einem **Dictionary**.

Zusammen fangen sie *alle* möglichen Argumente ab, die eine Funktion haben könnte, und machen unseren Dekorator universell einsetzbar.

---

### **Ein universeller `@debug_info`-Dekorator**

Dieser Dekorator gibt den Namen der aufgerufenen Funktion und alle ihre Argumente aus.

In [29]:
def debug_info(func):
    # Der Wrapper akzeptiert nun *args und **kwargs
    def wrapper(*args, **kwargs):
        print(f"[DEBUG] Aufruf von Funktion '{func.__name__}'...")
        print(f"        Positions-Argumente (args): {args}")
        print(f"        Keyword-Argumente (kwargs): {kwargs}")
        
        # Die Argumente werden an die Originalfunktion durchgereicht
        ergebnis = func(*args, **kwargs)
        
        print(f"[DEBUG] Funktion '{func.__name__}' beendet. Rückgabewert: {ergebnis}")
        return ergebnis
    return wrapper

@debug_info
def addiere(a, b):
    return a + b

@debug_info
def gruss(name, anrede="Hallo"):
    return f"{anrede}, {name}!"

# Tests
addiere(5, 3)
print("-" * 30)
gruss("Welt", anrede="Guten Tag")

[DEBUG] Aufruf von Funktion 'addiere'...
        Positions-Argumente (args): (5, 3)
        Keyword-Argumente (kwargs): {}
[DEBUG] Funktion 'addiere' beendet. Rückgabewert: 8
------------------------------
[DEBUG] Aufruf von Funktion 'gruss'...
        Positions-Argumente (args): ('Welt',)
        Keyword-Argumente (kwargs): {'anrede': 'Guten Tag'}
[DEBUG] Funktion 'gruss' beendet. Rückgabewert: Guten Tag, Welt!


'Guten Tag, Welt!'

---

### **Umgang mit Rückgabewerten**

Wie im `@debug_info`-Beispiel gesehen, ist es entscheidend, dass der `wrapper` den Rückgabewert der Originalfunktion `func` auffängt und selbst zurückgibt (`return ergebnis`). Andernfalls würde der Dekorator den Rückgabewert "verschlucken" und die dekorierte Funktion würde immer `None` zurückgeben.

---

In [30]:
import time

def timer_dekorator(func):
    # Der Wrapper akzeptiert alle möglichen Argumente
    def wrapper(*args, **kwargs): 
        start_zeit = time.time()
        ergebnis = func(*args, **kwargs) # Aufruf der Originalfunktion
        end_zeit = time.time()
        print(f"[Timer] Funktion '{func.__name__}' hat {end_zeit - start_zeit:.4f} Sekunden gedauert.")
        return ergebnis
    return wrapper

@timer_dekorator
def lange_berechnung(n):
    """Eine Funktion, die eine Weile beschäftigt ist."""
    summe = 0
    for i in range(n):
        summe += i
    return summe

@timer_dekorator
def gruss_mit_verzoegerung(name):
    """Eine Funktion, die kurz wartet."""
    print(f"Hallo, {name}!")
    time.sleep(1)

# Aufruf der dekorierten Funktionen
lange_berechnung(10000000)
gruss_mit_verzoegerung("Welt")

@timer_dekorator
def berechne_fakultaet(n):
    """Berechnet die Fakultät einer Zahl (sehr ineffizient)."""
    ergebnis = 1
    for i in range(1, n + 1):
        ergebnis *= i
    return ergebnis

fakultaet_von_15 = berechne_fakultaet(15)
print(f"Das Ergebnis der Berechnung ist: {fakultaet_von_15}")

[Timer] Funktion 'lange_berechnung' hat 0.1804 Sekunden gedauert.
Hallo, Welt!
[Timer] Funktion 'gruss_mit_verzoegerung' hat 1.0031 Sekunden gedauert.
[Timer] Funktion 'berechne_fakultaet' hat 0.0000 Sekunden gedauert.
Das Ergebnis der Berechnung ist: 1307674368000



**Analyse:** Der `@timer_dekorator` hat beiden Funktionen eine Zeitmessung hinzugefügt, ohne dass wir ihren internen Code ändern mussten. `*args` und `**kwargs` im Wrapper stellen sicher, dass der Dekorator mit jeder Funktion funktioniert, egal welche Argumente sie hat.


In [31]:
# Übung

---

### **Dekoratoren mit eigenen Argumenten**

Was, wenn wir dem Dekorator selbst einen Parameter übergeben wollen, z.B. `@repeat(3)`?

Dies erfordert eine zusätzliche Verschachtelungsebene. Es ist wie eine "Dekorator-Fabrik".

* **Ebene 1: Die Fabrik (`repeat`):** Eine Funktion, die die Argumente des Dekorators (z.B. `n=3`) entgegennimmt. Ihre einzige Aufgabe ist es, den eigentlichen Dekorator zurückzugeben.
* **Ebene 2: Der Dekorator (`dekorator_selbst`):** Dies ist die Funktion, die wir kennen. Sie nimmt die zu dekorierende Funktion (`func`) entgegen.
* **Ebene 3: Der Wrapper (`wrapper`):** Dies ist die Funktion, die am Ende ausgeführt wird. Sie hat Zugriff auf die Argumente aus **beiden** äußeren Ebenen (`n` aus der Fabrik und `func` aus dem Dekorator).

**Was Python tut, wenn es `@repeat(3)` sieht:**

1.  Es ruft zuerst `repeat(3)` auf.
2.  Die `repeat`-Funktion läuft und gibt die `dekorator_selbst`-Funktion zurück.
3.  Python nimmt diesen zurückgegebenen Dekorator und wendet ihn auf die darunter stehende Funktion an, als ob dort `@dekorator_selbst` stehen würde.

---

### **Isoliertes Beispiel 3: Der `@repeat(n)`-Dekorator**



In [32]:
def repeat(n):
    # Ebene 1: Die Fabrik, die das Argument 'n' entgegennimmt
    def dekorator_selbst(func):
        # Ebene 2: Der eigentliche Dekorator
        def wrapper(*args, **kwargs):
            # Ebene 3: Der Wrapper, der die Logik ausführt
            print(f"Funktion wird {n}-mal wiederholt:")
            for _ in range(n):
                func(*args, **kwargs)
        return wrapper
    return dekorator_selbst

@repeat(3)
def feuer_laser():
    print("-> Laserstrahl abgefeuert!")

# Aufruf der dekorierten Funktion
feuer_laser()

Funktion wird 3-mal wiederholt:
-> Laserstrahl abgefeuert!
-> Laserstrahl abgefeuert!
-> Laserstrahl abgefeuert!


---
### **Transfer auf das Projekt: Ein `@check_energie`-Dekorator für das `Raumschiff`**

Jetzt wenden wir das Gelernte auf unser Projekt an. Wir wollen einen Dekorator schreiben, der vor der Ausführung einer Methode prüft, ob das Raumschiff genug Energie hat. Dies ist ein praktisches Beispiel für einen Dekorator mit Zustand (er greift auf `self` zu).

In [33]:
from functools import wraps

def benoetigt_energie(energie_kosten):
    """Dekorator-Fabrik, die prüft, ob genug Energie vorhanden ist."""
    def dekorator(methode):
        @wraps(methode) # wraps kopiert Metadaten wie den Namen der Originalfunktion
        def wrapper(self, *args, **kwargs):
            if self.energie_level >= energie_kosten:
                print(f"[ENERGIE-CHECK] Ausreichend Energie ({self.energie_level} Einheiten) vorhanden.")
                self.energie_level -= energie_kosten
                return methode(self, *args, **kwargs)
            else:
                print(f"[ENERGIE-CHECK] FEHLER: Nicht genug Energie! Benötigt: {energie_kosten}, Vorhanden: {self.energie_level}.")
                return None
        return wrapper
    return dekorator


class Raumschiff:
    def __init__(self, name, energie_level):
        self.name = name
        self.energie_level = energie_level

    @benoetigt_energie(10)
    def warpantrieb_aktivieren(self):
        print(f"'{self.name}' springt auf Warp-Geschwindigkeit!")

    @benoetigt_energie(50)
    def schilde_hochfahren(self):
        print(f"'{self.name}' aktiviert die Deflektorschilde!")
        
    def __str__(self):
        return f"Schiff: {self.name}, Energie: {self.energie_level}"

# Test
enterprise = Raumschiff("Enterprise", 100)
print(enterprise)

enterprise.schilde_hochfahren()
print(enterprise)

enterprise.warpantrieb_aktivieren()
print(enterprise)

# Dieser Aufruf wird fehlschlagen
enterprise.schilde_hochfahren()
print(enterprise)

Schiff: Enterprise, Energie: 100
[ENERGIE-CHECK] Ausreichend Energie (100 Einheiten) vorhanden.
'Enterprise' aktiviert die Deflektorschilde!
Schiff: Enterprise, Energie: 50
[ENERGIE-CHECK] Ausreichend Energie (50 Einheiten) vorhanden.
'Enterprise' springt auf Warp-Geschwindigkeit!
Schiff: Enterprise, Energie: 40
[ENERGIE-CHECK] FEHLER: Nicht genug Energie! Benötigt: 50, Vorhanden: 40.
Schiff: Enterprise, Energie: 40


---

## **3. `@property`: Der Pythonic Weg zur Kapselung**

### **Theorie: Weg von `get_...` und `set_...`**

In vielen Programmiersprachen ist es üblich, für den Zugriff auf Attribute Methoden wie `get_gehalt()` und `set_gehalt(wert)` zu schreiben. Das geht in Python auch, fühlt sich aber umständlich an.

**Das Problem:** Wir wollen den direkten Zugriff `obj.gehalt` beibehalten, aber trotzdem eine Logik ausführen, wenn das Attribut gelesen oder geschrieben wird (z.B. eine Validierung).

**Die Lösung:** Der eingebaute **`@property`**-Dekorator. Er verwandelt eine Methode in ein "verwaltetes Attribut". Er ermöglicht es uns, Getter-, Setter- und Deleter-Logik zu definieren.

---

### **Read-Only Property**

Wir definieren eine `Temperatur`-Klasse, die den Wert intern immer in Celsius speichert, aber eine bequeme, schreibgeschützte Eigenschaft zur Abfrage in Fahrenheit anbietet.

In [34]:
class Temperatur:
    def __init__(self, celsius):
        self._celsius = celsius # "_"-Präfix signalisiert: "interne Variable"

    @property
    def fahrenheit(self):
        # Diese Methode wird aufgerufen, wenn wir '.fahrenheit' abfragen
        return (self._celsius * 9/5) + 32

t = Temperatur(25)
print(f"{t._celsius}°C sind {t.fahrenheit}°F.")

# Versuch, die Fahrenheit-Eigenschaft zu schreiben -> Führt zu einem Fehler!
# t.fahrenheit = 77 # AttributeError: can't set attribute

25°C sind 77.0°F.



---

### **Property mit Getter und Setter**

Jetzt erlauben wir auch das Schreiben auf ein Attribut, aber fangen die Zuweisung mit einer Validierungslogik ab.

In [35]:
class Mitarbeiter:
    def __init__(self, name, gehalt):
        self.name = name
        self._gehalt = gehalt # Internes Attribut
    
    # Der Getter: Wird bei 'obj.gehalt' aufgerufen
    @property
    def gehalt(self):
        print("-> Getter für Gehalt wird aufgerufen...")
        return self._gehalt

    # Der Setter: Wird bei 'obj.gehalt = wert' aufgerufen
    @gehalt.setter
    def gehalt(self, neuer_wert):
        print(f"-> Setter wird versucht mit Wert {neuer_wert}...")
        if neuer_wert < 0:
            print("FEHLER: Gehalt kann nicht negativ sein!")
        else:
            self._gehalt = neuer_wert

m = Mitarbeiter("Ben", 50000)

# Ruft den Getter auf
print(f"Bens Gehalt: {m.gehalt}")

# Ruft den Setter auf
m.gehalt = 55000 
print(f"Neues Gehalt: {m.gehalt}")

print("-" * 20)
# Versuch einer ungültigen Zuweisung, der Setter verhindert es
m.gehalt = -1000
print(f"Gehalt nach ungültigem Versuch: {m.gehalt}")

-> Getter für Gehalt wird aufgerufen...
Bens Gehalt: 50000
-> Setter wird versucht mit Wert 55000...
-> Getter für Gehalt wird aufgerufen...
Neues Gehalt: 55000
--------------------
-> Setter wird versucht mit Wert -1000...
FEHLER: Gehalt kann nicht negativ sein!
-> Getter für Gehalt wird aufgerufen...
Gehalt nach ungültigem Versuch: 55000


In [36]:
# Übung


---

### **Transfer auf das Projekt: Kontrollierte `Raumschiff`-Attribute**

Wir nutzen `@property`, um die `huelle_staerke` unseres Raumschiffs zu einem verwalteten Attribut zu machen. Es soll nicht möglich sein, die Hülle über 100% zu "reparieren".

In [37]:
class Raumschiff:
    MAX_HUELLE = 100
    
    def __init__(self, name, huelle_staerke):
        self.name = name
        # Wichtig: Wir weisen hier dem privaten Attribut zu,
        # um den Setter beim ersten Mal zu umgehen.
        self._huelle_staerke = huelle_staerke

    @property
    def huelle_staerke(self):
        return self._huelle_staerke

    @huelle_staerke.setter
    def huelle_staerke(self, wert):
        if wert > self.MAX_HUELLE:
            self._huelle_staerke = self.MAX_HUELLE
            print(f"Hülle kann nicht über {self.MAX_HUELLE}% sein. Setze auf Maximum.")
        elif wert < 0:
            self._huelle_staerke = 0
        else:
            self._huelle_staerke = wert
            
# Testen
enterprise = Raumschiff("Enterprise-D", 50)
print(f"Aktuelle Hülle: {enterprise.huelle_staerke}%")

print("\nRepariere um 80 Punkte...")
enterprise.huelle_staerke += 80 # Ruft Getter und dann Setter auf!
print(f"Neue Hülle: {enterprise.huelle_staerke}%")

Aktuelle Hülle: 50%

Repariere um 80 Punkte...
Hülle kann nicht über 100% sein. Setze auf Maximum.
Neue Hülle: 100%



---
---

## **4. `@classmethod` und `@staticmethod`**

### **Werkzeuge für die Klasse, nicht die Instanz**

Nachdem wir Dekoratoren verstehen, können wir diese beiden wichtigen eingebauten Dekoratoren neu betrachten. Sie verändern, wie eine Methode an die Klasse oder Instanz gebunden ist.

**`@classmethod`**
* **Parameter:** Erhält als ersten Parameter automatisch die **Klasse selbst**, konventionell `cls` genannt.
* **Zweck:** Wird verwendet, um Methoden zu erstellen, die auf der Klasse operieren, nicht auf einer spezifischen Instanz. Der häufigste Anwendungsfall sind **alternative Konstruktoren**.

**`@staticmethod`**
* **Parameter:** Erhält **keinen** automatischen ersten Parameter (weder `self` noch `cls`).
* **Zweck:** Im Grunde eine normale Funktion, die aber im "Namensraum" der Klasse lebt, weil sie thematisch dorthin gehört. Sie kann weder den Zustand der Klasse (`cls`) noch den der Instanz (`self`) ändern. Man nutzt sie für **Utility-Funktionen**.

---

### **Eine `Datum`-Klasse**

In [38]:
import datetime

class Datum:
    def __init__(self, tag, monat, jahr):
        self.tag = tag
        self.monat = monat
        self.jahr = jahr
        
    def __str__(self):
        return f"{self.tag:02d}.{self.monat:02d}.{self.jahr}"

    # Ein alternativer Konstruktor
    @classmethod
    def heute(cls):
        # 'cls' ist hier die Klasse 'Datum'
        heute = datetime.date.today()
        # Ruft den normalen __init__ auf, um eine Instanz zu erstellen
        return cls(heute.day, heute.month, heute.year)

    # Eine Utility-Funktion, die keinen Zustand braucht
    @staticmethod
    def ist_schaltjahr(jahr):
        return jahr % 4 == 0 and (jahr % 100 != 0 or jahr % 400 == 0)

# Normaler Konstruktor
geburtstag = Datum(5, 10, 1990)
print(f"Geburtstag: {geburtstag}")

# Alternativer Konstruktor via @classmethod
heutiges_datum = Datum.heute()
print(f"Heute ist der: {heutiges_datum}")

# Utility-Funktion via @staticmethod
print(f"War 2024 ein Schaltjahr? {Datum.ist_schaltjahr(2024)}")
print(f"War 1990 ein Schaltjahr? {Datum.ist_schaltjahr(1990)}")

Geburtstag: 05.10.1990
Heute ist der: 29.06.2025
War 2024 ein Schaltjahr? True
War 1990 ein Schaltjahr? False


# Übung


---
### **Transfer auf das Projekt: `Raumschiff`-Fabriken und -Validatoren**

In [39]:
class Raumschiff:
    def __init__(self, name, klasse, huelle):
        if not self.validiere_kennung(name):
             raise ValueError("Ungültige Schiffskennung!")
        self.name = name
        self.klasse = klasse
        self.huelle = huelle
    
    def __str__(self):
        return f"{self.klasse}-Klasse '{self.name}' (Hülle: {self.huelle}%)"
    
    @classmethod
    def baue_standard_fregatte(cls, name):
        # Ruft den normalen __init__ mit vordefinierten Werten auf
        return cls(name, "Fregatte", 100)

    @staticmethod
    def validiere_kennung(kennung):
        # Eine Utility-Funktion, die prüft, ob eine Kennung gültig ist
        return kennung.startswith("USS-") and len(kennung) > 4

# Erstellen via @classmethod
fregatte = Raumschiff.baue_standard_fregatte("USS-Reliant")
print(f"Neue Fregatte gebaut: {fregatte}")

# Nutzen der @staticmethod
is_valid = Raumschiff.validiere_kennung("NCC-1701")
print(f"Ist 'NCC-1701' eine gültige Kennung? {is_valid}")

# Versuch, ein Schiff mit ungültiger Kennung zu bauen
# ungueltiges_schiff = Raumschiff("Defiant", "Eskorte", 120) # -> ValueError

Neue Fregatte gebaut: Fregatte-Klasse 'USS-Reliant' (Hülle: 100%)
Ist 'NCC-1701' eine gültige Kennung? False



---
---

## **5. Tages-Challenge: Anwendung im Schüler-Projekt (`Smart Grid`)**

Jetzt kombinieren wir alle heutigen Konzepte in Ihrem Projekt.

**Ihre umfassende Aufgabe:**

1.  **Schreiben Sie einen Dekorator:**
    * Erstellen Sie einen Dekorator `@log_leistungsanpassung`, der vor und nach dem Aufruf einer Methode eine Log-Nachricht ausgibt.
    * Beispiel: `"[LOG] Leistung wird angepasst..."` und `"[LOG] Anpassung abgeschlossen."`.
2.  **Dekorator anwenden:**
    * Fügen Sie Ihrer `Energieerzeuger`-Klasse (oder einer der Kindklassen) eine Methode `kalibrieren()` hinzu, die nur eine Nachricht ausgibt (`"System wird kalibriert..."`).
    * Wenden Sie Ihren `@log_leistungsanpassung`-Dekorator auf diese `kalibrieren()`-Methode an.
3.  **`@property` implementieren:**
    * Das Attribut `aktuelle_leistung` in Ihrem `Energieerzeuger` soll zu einer verwalteten Eigenschaft werden.
    * Der **Getter** soll einfach den Wert zurückgeben.
    * Der **Setter** soll sicherstellen, dass die `aktuelle_leistung` niemals größer als die `max_leistung` oder kleiner als 0 sein kann. Bei ungültigen Werten soll eine Warnung ausgegeben und der Wert auf den nächstgelegenen gültigen Wert (0 oder `max_leistung`) gesetzt werden.
4.  **`@classmethod` und `@staticmethod` hinzufügen:**
    * Fügen Sie Ihrer `Solaranlage`-Klasse eine `@classmethod` namens `erzeuge_standard_farm(name)` hinzu. Diese Methode soll eine `Solaranlage` mit vordefinierten Werten für `max_leistung` (z.B. 50 MW) und `panel_flaeche` (z.B. 20000 m²) erstellen und zurückgeben.
    * Fügen Sie Ihrer Basisklasse `Energieerzeuger` eine `@staticmethod` namens `ist_netz_stabil(frequenz)` hinzu. Sie soll `True` zurückgeben, wenn die übergebene `frequenz` zwischen 49.8 und 50.2 Hz liegt, andernfalls `False`.
5.  **Alles testen:**
    * Rufen Sie Ihre dekorierte `kalibrieren()`-Methode auf.
    * Versuchen Sie, die `aktuelle_leistung` auf einen gültigen und einen ungültigen Wert zu setzen.
    * Erstellen Sie eine neue Farm mit Ihrer `@classmethod`.
    * Testen Sie die Netzstabilität mit Ihrer `@staticmethod`.
```



---
---

## **Inhalt für die Übungsdateien von Tag 4**

---
### **Inhalt für `Uebung_4_Block_1.ipynb`**
````markdown
# Übungen zu Block 1: Eigene Dekoratoren bauen

---
**Aufgabe 1: Ein einfacher "Hallo"-Dekorator**
Schreiben Sie einen Dekorator namens `@hallo_sagen`, der vor dem Aufruf der dekorierten Funktion einfach "Hallo!" ausgibt.

Testen Sie ihn an einer Funktion `tue_etwas()`, die "Ich tue etwas Wichtiges." ausgibt.

```python
# Ihr Code hier
```
---
**Aufgabe 2: Argumente akzeptieren**
Erweitern Sie Ihren Dekorator, sodass er auch mit Funktionen funktioniert, die Argumente entgegennehmen. Nutzen Sie `*args` und `**kwargs` im `wrapper`.

Testen Sie ihn an einer neuen Funktion `addiere(a, b)`, die zwei Zahlen addiert und das Ergebnis zurückgibt. Der Dekorator soll das Ergebnis der Originalfunktion korrekt zurückgeben.

```python
# Ihr Code hier
```
---
**Aufgabe 3: Ein praktischer `@debug`-Dekorator**
Erstellen Sie einen Dekorator `@debug`, der vor der Ausführung einer Funktion deren Namen und die übergebenen Argumente ausgibt.

*Beispiel-Ausgabe:* `--- Aufruf von 'multipliziere' mit args=(10, 3), kwargs={} ---`

Testen Sie ihn an der Funktion `multipliziere(x, y)`

```python
# Ihr Code hier
```
---
**Aufgabe 4 (Fortgeschritten): Ein Dekorator, der selbst ein Argument annimmt**
Erstellen Sie einen Dekorator `@repeat(n)`, der die dekorierte Funktion `n`-mal hintereinander ausführt.

*Tipp:* Sie benötigen hier eine dreifache Funktions-Verschachtelung: `dekorator(n) -> wrapper(func) -> inner_wrapper(*args, **kwargs)`.

Testen Sie mit `@repeat(3)` auf einer Funktion, die "Wiederholung!" ausgibt.

```python
# Ihr Code hier
```